In [1]:
from importlib import reload
import flymovie as fm
from flymovie.simnuc import Sim
import matplotlib.pyplot as plt
import numpy as np
import os
import random
import pickle
from pathlib import Path
import scipy.ndimage as ndi
from importlib import reload
import scipy
import pandas as pd
import scipy
from copy import deepcopy

%load_ext autoreload
%autoreload 2

In [5]:
outfolder = '/Users/michaelstadler/Bioinformatics/Projects/rpb1/results/batch_test'
fm.sim_rpb1_batch(outfolder, kernel_real, 2,1,1,(200,200,200), nuc_rad=92, nfree_rng=[10_000,50_000], hlb_diam_rng=[7,15], 
    hlb_nmols_rng=[100,1000], n_clusters_rng=[0,300], cluster_diam_mean_rng=[1,3], 
    cluster_diam_var_rng=[0.5, 2], cluster_nmols_mean_rng=[10,100], cluster_nmols_var_rng=[1,10], noise_sigma_rng=[1_000, 2_000])


arglist done


In [80]:
def rebin(arr, new_shape):
    """Bin a 2D numpy array to fit a new shape, with each entry
    in the binned array equal to the sum of the constituent
    entries in the original."""
    shape = ()
    for i in range(arr.ndim):
        shape = shape + (new_shape[i], arr.shape[i] // new_shape[i],)

    binned = arr.reshape(shape)
    for i in range(arr.ndim):
        binned = binned.sum(i + 1)
    
    return binned

## Tuning to real data

In [5]:
def norm(x):
    return (x - np.min(x)) / (np.max(x) - np.min(x))

In [4]:
realstack = fm.read_czi('/Users/michaelstadler/Bioinformatics/Projects/rpb1/data/counting_blobs/cycle 12/embryo17.czi', swapaxes=False)

In [2]:
kernel2 = fm.load_pickle('/Users/michaelstadler/Bioinformatics/Projects/rpb1/PSFs/psf_20220210_21x25x25pixels_100x50x50voxel.pkl')

In [6]:
sub = fm.stack_normalize_minmax(realstack[:,350:, 100:300]) * 1000
sub = sub[:,0:100, 25:125]

In [11]:
mask = Sim.make_spherical_mask(100,100,100, 40)
sim = Sim(mask, 85, 85)
sim.add_kernel(kernel2, 100, 50)
# Free
sim.add_n_objects(10_000, 100, 1, 1, 'nuc')
# HLB
sim.add_sphere((47,35,35), 100, 400, 5)
sim.add_sphere((47,50,68), 100, 400, 5)
# clusters
sim.add_n_objects(60, 100, 13, 2)
sim.add_noise('poisson')
sim.convolve()
sim.resize((250,85,85), order=1)
sim.im = fm.stack_normalize_minmax(sim.im) * 1000
sim.add_noise('gaussian', sigma=27)
fm.viewer([sim.im * (np.mean(sub)/np.mean(sim.im)), sub], 5)


interactive(children=(Dropdown(description='Color', index=7, options=('inferno', 'gray_r', 'Gators', 'magma', …

In [11]:
sim.im.shape

(34, 100, 100)

In [22]:
np.std(sub[0])

15.511245459892377

In [5]:
im = fm.load_pickle('/Users/michaelstadler/Bioinformatics/Projects/rpb1/results/testsimsoThRIpoU/ErD_20914_13.52_540_200_3_4.99_32_3.7_13.6_rep1.pkl')
fm.viewer(im.max(axis=0), 5)

interactive(children=(Dropdown(description='Color', index=4, options=('gray_r', 'Gators', 'gray', 'magma', 'ci…